# **Project : A Case Study of ExpressWay Logistics**

**Business Overview:**

ExpressWay Logistics is a dynamic logistics service provider, committed to delivering efficient, reliable and cost-effective courier transportation and warehousing solutions. With a focus on speed, precision and customer satisfaction, we aim to be the go-to partner for our customers seeking seamless courier services. Our core service involves ensuring operational efficiency throughout our delivery and courier services, including inventory management, durable packaging and swift dispatch of couriers, real time tracking of shipments and on-time delivery of couriers as promised. We are committed to enhance our logistics and courier services and improve seamless connectivity for our customers.

**Current Challenge:**

ExpressWay Logistics faces numerous challenges in ensuring seamless deliveries and customer satisfaction. These challenges include managing various customer demands simultaneously, addressing delays in deliveries and ensuring products arrive intact and safe. Additionally, the company struggles with complexity of efficiently storing and handling a large volume of packages and ultimately meeting customer expectations. Moreover, maintaining a skilled workforce capable of handling various aspects of logistics operations presents its own set of challenges. Overcoming these obstacles requires a comprehensive approach that integrates innovative technology, strategic planning, and continuous improvement initiatives to ensure smooth operations and exceptional service delivery.

**Objective:**

Our primary objective is to conduct a sentiment analysis of user-generated reviews across various digital channels and platforms. By paying attention to their feedback, we want to find ways to make our services better - like handling different customer demands simultaneously, dealing with late deliveries, and keeping packages secured and intact. Through the application of prompt engineering methodologies and sentiment analysis, we'll figure out if sentiments expressed by users for our courier services are Positive or Negative. This will help us understand where we need to improve in order to meet customer expectations and keep them happy. With a focus on getting better all the time, we'll overcome the challenges at ExpressWay Logistics and make our services the best.

**Data Description:**

The dataset titled "courier-service_reviews.csv" is structured to facilitate sentiment analysis for courier service reviews. Here's a brief description of the data columns:

1. id: This column contains unique identifiers for each review entry. It helps in distinguishing and referencing individual reviews.
2. review: This column includes the actual text of the courier service reviews. The reviews are likely composed of customer opinions and experiences regarding different aspects of the services provided by ExpressWay Logistics.
3. sentiment: This column provides an additional layer of classification (positive and negative) for the mentioned reviews.

### Installation

In [1]:
!pip install openai==1.2 tiktoken datasets session-info --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.9/219.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00


### Imports

In [2]:
# Import all Python packages required to access the Azure Open AI API.
# Import additional packages required to access datasets and create examples.

from openai import AzureOpenAI
import json
import random
import tiktoken
import session_info

import pandas as pd
import numpy as np

from collections import Counter
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tabulate import tabulate

In [3]:
session_info.show(excludes= ["backports"])

### Authentication

In [4]:
with open('config.json', 'r') as az_creds:
    data = az_creds.read()

In [7]:
creds = json.loads(data)

In [8]:
client = AzureOpenAI(
    azure_endpoint=creds["AZURE_OPENAI_ENDPOINT"],
    api_key=creds["AZURE_OPENAI_KEY"],
    api_version=creds["AZURE_OPENAI_APIVERSION"]
)

In [9]:
chat_model_id = creds["CHATGPT_MODEL"]

### Utilities

In [10]:
def num_tokens_from_messages(messages):

    """
    Return the number of tokens used by a list of messages.
    Adapted from the Open AI cookbook token counter
    """

    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

    # Each message is sandwiched with <|start|>role and <|end|>
    # Hence, messages look like: <|start|>system or user or assistant{message}<|end|>

    tokens_per_message = 3 # token1:<|start|>, token2:system(or user or assistant), token3:<|end|>

    num_tokens = 0

    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))

    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>

    return num_tokens

## Task : Sentiment Analysis

In [26]:
 cs_reviews_df = pd.read_csv("courier-service_reviews.csv")


In [27]:
cs_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         131 non-null    int64 
 1   review     131 non-null    object
 2   sentiment  131 non-null    object
dtypes: int64(1), object(2)
memory usage: 3.2+ KB


In [28]:
cs_reviews_df['sentiment']= cs_reviews_df['sentiment'].str.lower()
cs_reviews_df['review']=cs_reviews_df['review'].str.lower()

In [29]:
cs_reviews_df.sample(5)

,id,review,sentiment
124,125,i encountered numerous issues with expressway ...,negative
54,55,the delivery executive from expressway logisti...,negative
42,43,i encountered numerous issues with expressway ...,negative
127,128,while expressway logistics' pricing is competi...,negative
111,112,i had a disappointing experience with expressw...,negative


In [30]:
cs_reviews_df.sentiment.value_counts()

,count
sentiment,
positive,68
negative,63


In [31]:
cs_reviews_df.shape

(131, 3)

Split The Dataset

In [32]:
cs_examples_df, cs_gold_examples_df = train_test_split(
    cs_reviews_df, #<- the full dataset
    test_size=0.2, #<- 20% random sample selected for gold examples
    random_state=42 #<- ensures that the splits are the same for every session
)

In [33]:
(cs_examples_df.shape, cs_gold_examples_df.shape)

((104, 3), (27, 3))

To select gold examples for this session, we sample randomly from the test data using a `random_state=42`. This ensures that the examples from multiple runs of the sampling are the same (i.e., they are randomly selected but do not change between different runs of the notebook). Note that we are doing this only to keep execution times low for illustration. In practise, large number of gold examples facilitate robust estimates of model accuracy.

In [34]:
columns_to_select = ['review','sentiment']

In [35]:
gold_examples = (
        cs_gold_examples_df.loc[:, columns_to_select]
                                     .sample(21, random_state=42) #<- ensures that gold examples are the same for every session
                                     .to_json(orient='records')
)

In [36]:
gold_examples

'[{"review":"the delivery executive assigned by expressway logistics was courteous and professional during the delivery process. they tried their best to handle the package with care.unfortunately, the package arrived with slight damage despite the delivery executive\'s efforts. the packaging seemed more than adequate to protect the contents during transit.","sentiment":"positive"},{"review":"expressway logistics failed to meet my expectations. the delivery was delayed, and the customer support team was unresponsive and unhelpful when i tried to inquire about the status of my parcel.","sentiment":"negative"},{"review":"expressway logistics\' incompetence resulted in a major inconvenience when my package was delivered to the wrong recipient. despite providing accurate delivery information, the package ended up in the hands of someone else, and efforts to retrieve it were unsuccessful. when i contacted customer service for assistance, i was met with apathy and a lack of urgency. their fa

In [37]:
json.loads(gold_examples)[0]     #Json format

{'review': "the delivery executive assigned by expressway logistics was courteous and professional during the delivery process. they tried their best to handle the package with care.unfortunately, the package arrived with slight damage despite the delivery executive's efforts. the packaging seemed more than adequate to protect the contents during transit.",
 'sentiment': 'positive'}

Derive Prompt

In [38]:
user_message_template = """```{courier_service_review}```"""

(A) Write Zero Shot System Message

In [39]:
zero_shot_system_message = """
Classify the following review presented in the input into one of the following sentiment.
sentiment - ['negative', 'positive']
review will be delimited by triple backticks in the input.
Answer only 'negative' or 'positive'. Nothing Else. Do not explain your answer.
"""


Create Zero Shot Prompt

In [40]:
zero_shot_prompt = [{'role':'system', 'content': zero_shot_system_message}]


In [41]:
num_tokens_from_messages(zero_shot_prompt)

63

 Write Few Shot System Message

In [42]:
few_shot_system_message = """
Classify the following review presented in the input into one of the following sentiment.
sentiment - ['negative', 'positive']
review will be delimited by triple backticks in the input.
Answer only 'negative' or 'positive'. Nothing Else. Do not explain your answer.
"""

Merely selecting random samples from the polarity subsets is not enough because the examples included in a prompt are prone to a set of known biases such as:
 - Majority label bias (frequent answers in predictions)
 - Recency bias (examples near the end of the prompt)


To avoid these biases, it is important to have a balanced set of examples that are arranged in random order. Let us create a Python function that generates bias-free examples:

In [43]:
def create_examples(dataset, n=4):

    """
    Return a JSON list of randomized examples of size 2n with two classes.
    Create subsets of each class, choose random samples from the subsets,
    merge and randomize the order of samples in the merged list.
    Each run of this function creates a different random sample of examples
    chosen from the training data.

    Args:
        dataset (DataFrame): A DataFrame with examples (review + label)
        n (int): number of examples of each class to be selected

    Output:
        randomized_examples (JSON): A JSON with examples in random order
    """

    positive_reviews = (dataset.sentiment == 'positive')
    negative_reviews = (dataset.sentiment == 'negative')
    columns_to_select = ['review', 'sentiment']

    positive_examples = dataset.loc[positive_reviews, columns_to_select].sample(n)
    negative_examples = dataset.loc[negative_reviews, columns_to_select].sample(n)

    examples = pd.concat([positive_examples, negative_examples])

    # sampling without replacement is equivalent to random shuffling

    randomized_examples = examples.sample(2*n, replace=False)

    return randomized_examples.to_json(orient='records')

Create Examples For Few shot prompt

In [44]:
examples = create_examples(cs_examples_df, 2)


In [45]:
json.loads(examples)

[{'review': "expressway logistics promises efficient delivery, but their drivers seem careless and unprofessional. i've received damaged goods more than once, indicating a lack of proper handling procedures.",
  'sentiment': 'negative'},
 {'review': 'expressway logistics makes shipping hassle-free.their tracking system is top-notch, giving me peace of mind every step of the way.i highly recommend expressway logistics for all your shipping needs.',
  'sentiment': 'positive'},
 {'review': 'expressway logistics is known for its prompt delivery times.it ensured that my parcels arrive well ahead of schedule.i am happy with the service.',
  'sentiment': 'positive'},
 {'review': "expressway logistics presents itself as a modern logistics solution with a focus on customer satisfaction. however, my recent experiences have highlighted some notable drawbacks. while their website offers a seamless booking experience and their packaging options are diverse, i've encountered issues with the accuracy

With the examples in place, we can now assemble a few-shot prompt. Since we will be using the few-shot prompt several times during evaluation, let us write a function to create a few-shot prompt (the logic of this function is depicted below).

In [46]:
def create_prompt(system_message, examples, user_message_template):

    """
    Return a prompt message in the format expected by the Open AI API.
    Loop through the examples and parse them as user message and assistant
    message.

    Args:
        system_message (str): system message with instructions for sentiment analysis
        examples (str): JSON string with list of examples
        user_message_template (str): string with a placeholder for courier service reviews

    Output:
        few_shot_prompt (List): A list of dictionaries in the Open AI prompt format
    """

    few_shot_prompt = [{'role':'system', 'content': system_message}]

    for example in json.loads(examples):
        example_review = example['review']
        example_sentiment = example['sentiment']

        few_shot_prompt.append(
            {
                'role': 'user',
                'content': user_message_template.format(
                    courier_service_review=example_review
                )
            }
        )

        few_shot_prompt.append(
            {'role': 'assistant', 'content': f"{example_sentiment}"}
        )

    return few_shot_prompt

Create Few Shot Prompt

In [47]:
few_shot_prompt = create_prompt(
    few_shot_system_message,
    examples,
    user_message_template
)


In [48]:
few_shot_prompt

[{'role': 'system',
  'content': "\nClassify the following review presented in the input into one of the following sentiment.\nsentiment - ['negative', 'positive']\nreview will be delimited by triple backticks in the input.\nAnswer only 'negative' or 'positive'. Nothing Else. Do not explain your answer.\n"},
 {'role': 'user',
  'content': "```expressway logistics promises efficient delivery, but their drivers seem careless and unprofessional. i've received damaged goods more than once, indicating a lack of proper handling procedures.```"},
 {'role': 'assistant', 'content': 'negative'},
 {'role': 'user',
  'content': '```expressway logistics makes shipping hassle-free.their tracking system is top-notch, giving me peace of mind every step of the way.i highly recommend expressway logistics for all your shipping needs.```'},
 {'role': 'assistant', 'content': 'positive'},
 {'role': 'user',
  'content': '```expressway logistics is known for its prompt delivery times.it ensured that my parcel

In [49]:
num_tokens_from_messages(few_shot_prompt)

318

 Evaluate prompts



Now we have two sets of prompts that we need to evaluate using gold labels. Since the few-shot prompt depends on the sample of examples that was drawn to make up the prompt, we expect some variability in evaluation. Hence, we evaluate each prompt multiple times to get a sense of the average and the variation around the average.

To reiterate, a choice on the prompt should account for variability due to the choice of the random sample. To aid repeated evaluation, we assemble an evaluation function .

In [50]:
def evaluate_prompt(prompt, gold_examples, user_message_template):

    """
    Return the micro-F1 score for predictions on gold examples.
    For each example, we make a prediction using the prompt. Gold labels and
    model predictions are aggregated into lists and compared to compute the
    F1 score.

    Args:
        prompt (List): list of messages in the Open AI prompt format
        gold_examples (str): JSON string with list of gold examples
        user_message_template (str): string with a placeholder for courier service review

    Output:
        micro_f1_score (float): Micro-F1 score computed by comparing model predictions
                                with ground truth
    """

    model_predictions, ground_truths, review_texts = [], [], []

    for example in json.loads(gold_examples):
        gold_input = example['review']
        user_input = [
            {
                'role':'user',
                'content': user_message_template.format(courier_service_review=gold_input)
            }
        ]

        try:
            response = client.chat.completions.create(
                model=chat_model_id,
                messages=prompt+user_input,
                temperature=0, # <- Note the low temperature (For a deterministic response)
                max_tokens=2 # <- Note how we restrict the output to not more than 2 tokens
            )

            prediction = response.choices[0].message.content
            # response = openai.ChatCompletion.create(
            #     deployment_id=chat_model_id,
            #     messages=prompt+user_input,
            #     temperature=0, # <- Note the low temperature(For a deterministic response)
            #     # max_tokens=2 # <- Note how we restrict the output to not more than 2 tokens
            # )

            # prediction = response['choices'][0]['message']['content']
            model_predictions.append(prediction.strip()) # <- removes extraneous white spaces
            ground_truths.append(example['sentiment'])
            review_texts.append(gold_input)

        except Exception as e:
            continue

    micro_f1_score = f1_score(ground_truths, model_predictions, average="micro")

    table_data = [[text, pred, truth] for text, pred, truth in zip(review_texts, model_predictions, ground_truths)]
    headers = ["Review", "Model Prediction", "Ground Truth"]
    print(tabulate(table_data, headers=headers, tablefmt="grid"))

    return micro_f1_score


Let us now use this function to do one evaluation of all the two prompts assembled so far, each time computing the Micro-F1 score.

Evaluate zero shot prompt

In [51]:
evaluate_prompt(zero_shot_prompt, gold_examples, user_message_template)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

0.9047619047619048

Evaluate few shot prompt

In [52]:
evaluate_prompt(few_shot_prompt, gold_examples, user_message_template)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

0.9047619047619048

However, this is just *one* choice of examples. We will need to run these evaluations with multiple choices of examples to get a sense of variability in F1 score for the few-shot prompt. As an example, let us run evaluations for the few-shot prompt 5 times.

In [53]:
num_eval_runs = 5

In [54]:
zero_shot_performance = []
few_shot_performance = []

In [55]:
for _ in tqdm(range(num_eval_runs)):

    # For each run create a new sample of examples
    examples = create_examples(cs_examples_df)

    # Assemble the zero shot prompt with these examples
    zero_shot_prompt = [{'role':'system', 'content': zero_shot_system_message}]
    # zero_shot_prompt = create_prompt(zero_shot_system_message, examples, user_message_template)

    # Assemble the few shot prompt with these examples
    few_shot_prompt = create_prompt(few_shot_system_message, examples, user_message_template)

    # Evaluate zero shot prompt accuracy on gold examples
    zero_shot_micro_f1 = evaluate_prompt(zero_shot_prompt, gold_examples, user_message_template)

    # Evaluate few shot prompt accuracy on gold examples
    few_shot_micro_f1 = evaluate_prompt(few_shot_prompt, gold_examples, user_message_template)

    zero_shot_performance.append(zero_shot_micro_f1)
    few_shot_performance.append(few_shot_micro_f1)

  0%|          | 0/5 [00:00<?, ?it/s]

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

 20%|██        | 1/5 [00:12<00:50, 12.66s/it]

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

 40%|████      | 2/5 [00:25<00:38, 12.74s/it]

...

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

 80%|████████  | 4/5 [01:02<00:17, 17.15s/it]

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      

100%|██████████| 5/5 [01:15<00:00, 15.16s/it]

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+----------------+
| Review                                                                                                                                                                                                                                                                                                                                                                                      



Compute the average (mean) and measure the variability (standard deviation) of the evaluation scores for both zero shot and few shot prompts.

In [56]:
zero_shot_performance_mean=np.round(np.mean(zero_shot_performance), 3)
zero_shot_performance_std=np.round(np.std(zero_shot_performance), 3)
print(f"Zero shot performance: {zero_shot_performance_mean} +/- {zero_shot_performance_std} ")
# Calculate for Zero Shot

Zero shot performance: 0.905 +/- 0.0 


In [57]:
few_shot_performance_mean=np.round(np.mean(few_shot_performance), 3)
few_shot_performance_std=np.round(np.std(few_shot_performance), 3)
print(f"Few shot performance: {few_shot_performance_mean} +/- {few_shot_performance_std} ")
# Calculate for Few Shot

Few shot performance: 0.905 +/- 0.0 
